In [ ]:
!pip install PyMuPDF

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install faiss-cpu

In [ ]:
#select a file from your computer to upload.
from google.colab import files
uploaded = files.upload()

In [ ]:
import fitz
#name your pdf document same as the name in the file uploaded.
pdf_document = "a brief history of time.pdf"
doc = fitz.open(pdf_document)

#this creates text chunks out of PDF.
text = ""
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text += page.get_text()

def split_text(text, max_length=500):
    sentences = text.split('. ')
    chunks = []
    current_chunk = []

    for sentence in sentences:
        if len(' '.join(current_chunk)) + len(sentence) <= max_length:
            current_chunk.append(sentence)
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks
text_chunks = split_text_with_overlap(text)

In [ ]:
#this will do embeddings
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
embeddings = model.encode(text_chunks)

In [ ]:
import faiss
import numpy as np

In [ ]:
embedding_matrix = np.array(embeddings)

index = faiss.IndexFlatL2(embedding_matrix.shape[1])

index.add(embedding_matrix)

faiss.write_index(index, "faiss_index.index")


In [ ]:
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer

model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [ ]:
# this code will take your queries, answer them until you press "q".
while(True):

  query = input("--->>>")
  if query == 'q':
    break
  query_embedding = model.encode([query])

  D, I = index.search(np.array(query_embedding), k=7)

  retrieved_chunks = [text_chunks[i] for i in I[0]]

  retrieved_context = " ".join(retrieved_chunks)
  # Check what context is retrieved by inserting print(retrieved_chunks) here.
  print(retrieved_context)

  input_text = f"Answer the question based on the following context: Question: {query}, Context: {retrieved_context} "
  inputs = tokenizer(input_text, return_tensors="pt")

  outputs = flan_model.generate(inputs["input_ids"], max_length=2000,num_beams=4)
  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print("Answer:", answer)
